In [1]:
import sys
sys.path.insert(0, '/tf/utils/')

In [2]:
import matplotlib.pyplot as plt
from IPython.display import Audio

In [3]:
from data_generators import NoisyTargetGenerator
from sound import Sound
from artigos.AttentionResUnet import TransformerUnet

In [4]:
base_shape_size = 8192
ws = 255
ol = 128
NORM_FACTOR = 128
input_shape = (128, 64, 2)

In [5]:
sound_base = Sound('../../Dados/Vozes/', '../../Dados/Ruido/', base_shape_size, limit=20000)

Loading Noise Files: 100%|██████████| 8137/8137 [00:05<00:00, 1383.38it/s]
/tf/utils/sound.py:65: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  clean_sounds = [sound for sound in clean_sounds if sound != self.TOO_SHORT_ERROR]
/tf/utils/sound.py:77: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  noise_sounds = [sound for sound in noise_sounds if sound != self.TOO_SHORT_ERROR]


In [6]:
data_generator_train = NoisyTargetGenerator(sound_base.train_X, sound_base.noise_sounds)
data_generator_val = NoisyTargetGenerator(sound_base.val_X, sound_base.noise_sounds)

In [7]:
model = TransformerUnet(input_shape).Transformer_UNet()

In [8]:
model.summary()

Model: "TransformerUNet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 64, 12  0           []                               
                                8)]                                                               
                                                                                                  
 conv2d (Conv2D)                (None, 128, 64, 32)  36896       ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 128, 64, 32)  128        ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                    

In [9]:
model.compile(optimizer='adam', loss='msle')

In [10]:
# Defina sua função de criação de gráfico com base nos dados de validação
def create_plot(x_val, y_val, model):
    F_f = model.predict(x_val, verbose=False)

    indice_aleatorio = np.random.choice(x_val.shape[0])
    x_test = x_val[indice_aleatorio]
    f_test = F_f[indice_aleatorio]
    y_test = y_val[indice_aleatorio]

    A = y_test[:, :, 0]
    phi = (y_test[:, :, 1] - 0.5) * 2 * np.pi
    
    A_f = f_test[:, :, 0]
    phi_f = (f_test[:, :, 1] - 0.5) * 2 * np.pi
    
    A_n = x_test[:, :, 0]
    phi_n = (x_test[:, :, 1] - 0.5) * 2 * np.pi

    s = reconstruct_signal_from_stft(A, phi)
    s_n = reconstruct_signal_from_stft(A_n, phi_n)
    s_f = reconstruct_signal_from_stft(A_f, phi_f)

    display.clear_output(wait=True)

    # Criando figuras e eixos separados para cada array
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1)
    
    # Plotando o primeiro array
    ax1.plot(s)
    ax1.set_ylabel('Sinal de voz ruidoso')
    
    # Plotando o primeiro array
    ax2.plot(s_n)
    ax2.set_ylabel('Sinal de voz ruidoso')
    
    ax3.plot(s_f)
    ax3.set_ylabel('Sinal de voz filtrado')
    
    # Exibindo os gráficos
    plt.show()

    # display_s = Audio(data=s, rate=8000)
    Audio(data=s, rate=8000)
    # display(display_s)
    # display_n = Audio(data=s_n, rate=8000)
    # display(display_n)
    # display_f = Audio(data=s_f, rate=8000)
    # display(display_f)
    
    fig, axs = plt.subplots(1, 3, figsize=(18, 6))  # Uma linha com três subplots

    # Primeiro gráfico
    im = axs[0].imshow(10*np.log10(A**2), aspect='auto', cmap='inferno')
    axs[0].set_title('Log Power Spectrum - Som original')
    axs[0].set_xlabel('Tempo (s)')
    axs[0].set_ylabel('Frequência (Hz)')
    fig.colorbar(im, ax=axs[0], format='%+2.0f dB')
    
    # Segundo gráfico
    im = axs[1].imshow(10*np.log10(A_n**2), aspect='auto', cmap='inferno')
    axs[1].set_title('Log Power Spectrum - Som ruidoso')
    axs[1].set_xlabel('Tempo (s)')
    axs[1].set_ylabel('Frequência (Hz)')
    fig.colorbar(im, ax=axs[1], format='%+2.0f dB')
    
    # Terceiro gráfico
    im = axs[2].imshow(10*np.log10(A_f**2), aspect='auto', cmap='inferno')
    axs[2].set_title('Log Power Spectrum - Sinal filtrado')
    axs[2].set_xlabel('Tempo (s)')
    axs[2].set_ylabel('Frequência (Hz)')
    fig.colorbar(im, ax=axs[2], format='%+2.0f dB')
    
    plt.tight_layout()  # Para evitar sobreposição de rótulos e gráficos
    plt.show()

In [11]:
batch_size = 64
steps_per_epoch = len(sound_base.train_X) // batch_size

print('Starting training')

for epoch in range(10):
    print(f"Epoch {epoch + 1}")
    
    # Gera um novo lote de validação para cada época
    validation_batch = next(data_generator_val.generate_sample_completo(batch_size=batch_size))
    x_val, y_val = validation_batch
    
    model.fit(data_generator_train.generate_sample_completo(batch_size=batch_size, include_clean=True),
                     steps_per_epoch=steps_per_epoch,
                     epochs=1,
                     validation_data=(x_val, y_val),
                    )
    create_plot(x_val, y_val, model)

Starting training
Epoch 1


InvalidArgumentError: Graph execution error:

Detected at node 'TransformerUNet/conv2d/Conv2D' defined at (most recent call last):
    File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/usr/local/lib/python3.8/dist-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/usr/local/lib/python3.8/dist-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/usr/local/lib/python3.8/dist-packages/ipykernel/kernelapp.py", line 725, in start
      self.io_loop.start()
    File "/usr/local/lib/python3.8/dist-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib/python3.8/asyncio/base_events.py", line 570, in run_forever
      self._run_once()
    File "/usr/lib/python3.8/asyncio/bas